# Match searches with bookings

- For every search in the searches file, find out if the search ended up in a booking or not(using the info in the bookings file). For instance, search and booking origin and destination should match. 
- For the bookings file, origin and destinatio are the columns dep_port and arr_port, respectively.
- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking and 0 otherwise. 

bookings --> dep_port, arr_port
searches --> origin, destination

No tengo columnas que relacionen una reserrva con una búsqueda. Son datos aislados. Además, se quiere saber que a lo mejor la búsqueda fue en Kayak, pero después ya lo compraste desde Iberia. Como lo ligo:?
- Por destino y origen.
- Por número de personas (aunque a veces la gente busca para 1, no busca para grupos)-
- Para familias, siempre suele funcionar (porque precio de niño es menor)
Puedo complicar el modelo todo lo que quiera, o todo lo que me pidan.

Para el ejercicio, tendremo sólo el criterio de que coincidadn en fecha

bookings --> dep_port, arr_port, cre_date
searches --> origin, destination, search_date
0 -> drop nan, add index
1-> make sure dep_port & arr_port dont have spaces
2 -> make suce cre_date and search_date have the same format
3 -> add 'Booked' column with 1 in bookings
4 -> group by dep_port, arr_port, cre_date
5 -> left join searches + Bookeds
6 -> Change Nan to 0 and if >0 to 1
7 -> Add column in the file.

Ojo, que si hemos borrado duplicados, no coincidirán los números de línea porque hemos borrado duplicados, quitado nans. Lo solución es conservar el index que Pandas nos proporciona en las datafranes. De ahí que en 0 se añada ese index

Como curiosidad, el Data Center de Amadeus tiene 
-Dos líneas de luz de compañías independientes por seguridad.
- Motores diesel por si aun así se va, .
etc

<img src="http://www.blueboxcooling.com/en/thumb/dettaglionews/data_centre2.jpg/>

##  Sobre los duplicados

Es habitual encontrarse datos de la mimsma búsqueda varias veces repetidas, por ejemplo que un robot ha hecho veinte veces la misma búsqueda dentro de una simple búsqueda de un usuario. 

Hay que preguntar a la empresa qué hacer con los duplicados. En este caso, las líneas son completamente iguales, lo ha hecho Israel. 

Pandas tiene una función drop duplicates que borra ese caso de una línea con todo repetido. 

El problema es que vamos a tener chunks, por lo que si no encontramos duplicados dentro de ese chunk no quiere decir que no lo haya, porque esté en otro chunk

# 1) Familiarizarse con datos

Ya hecho en anteriores ejercicios...

# 2) Preparar datos

In [11]:
import pandas as pd

In [5]:
cd ~/Data/challenge/

/home/dsc/Data/challenge


In [13]:
bookings = pd.read_csv('bookings.csv.bz2',sep="^", nrows=1200000)

In [14]:
bookings.shape

(1200000, 38)

In [15]:
bookings.drop_duplicates().shape

(1000000, 38)

####  Importante: esto siguiente se tendría que haber hecho con low_memory = False. Lo hicimos sin eso por error, pero como tarda en hacerse lo dejamos así. 

In [19]:
bookings = pd.read_csv('bookings.csv.bz2', sep='^', chunksize=300000)

all_chunks = pd.DataFrame()
for i, chunk in enumerate(bookings):
    all_chunks = all_chunks.append(chunk)aa
    all_chunks = all_chunks.drop_duplicates()
    print(f"{i}, Length: {len(all_chunks)}")

0, Length: 300000
1, Length: 600000
2, Length: 900000
3, Length: 1000000
4, Length: 1000000
5, Length: 1000000
6, Length: 1000000
7, Length: 1000000
8, Length: 1000000
9, Length: 1000000
10, Length: 1000000
11, Length: 1000000
12, Length: 1000000
13, Length: 1000000
14, Length: 1000000
15, Length: 1000000


/home/dsc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


16, Length: 1016377
17, Length: 1016377
18, Length: 1016377
19, Length: 1016377
20, Length: 1016377
21, Length: 1016377
22, Length: 1016377
23, Length: 1016377
24, Length: 1016377
25, Length: 1016377
26, Length: 1016377
27, Length: 1016377
28, Length: 1016377
29, Length: 1016377
30, Length: 1016377
31, Length: 1016377
32, Length: 1016377
33, Length: 1016377


In [20]:
all_chunks.to_csv('bookings_no_dup.csv', sep='^', index=False )

In [21]:
! bzip2 bookings_no_dup.csv
! ls -alt

total 1073864
drwxr-x--- 2 dsc dsc      4096 ene 26 12:02 .
-rw-r--r-- 1 dsc dsc  56437955 ene 26 12:01 bookings_no_dup.csv.bz2
-rwxrwxrwx 1 dsc dsc    244720 ene 25 17:20 searches_sample.csv.bz2
-rwxrwxrwx 1 dsc dsc   4232732 ene 25 17:19 bookings_sample.csv
-rwxrwxrwx 1 dsc dsc    535893 ene 25 17:19 bookings_sample.csv.bz2
-rwxr-x--- 1 dsc dsc 483188920 mar 13  2018 searches.csv.bz2
-rwxr-x--- 1 dsc dsc 554970628 mar 13  2018 bookings.csv.bz2
drwxr-x--- 7 dsc dsc      4096 mar 13  2018 ..


In [22]:
%%time
searches = pd.read_csv('searches.csv.bz2', sep='^', chunksize=300000)

all_chunks = pd.DataFrame()
for i, chunk in enumerate(searches):
    all_chunks = all_chunks.append(chunk)
    all_chunks = all_chunks.drop_duplicates()
    print(f"{i}, Length: {len(all_chunks)}")

<string>:2: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.


0, Length: 299999
1, Length: 365535
2, Length: 372071
3, Length: 375383


<string>:2: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.


4, Length: 393537
5, Length: 403277
6, Length: 403277
7, Length: 403277
8, Length: 404843
9, Length: 404843
10, Length: 409813
11, Length: 414701
12, Length: 414701
13, Length: 416267
14, Length: 416267
15, Length: 416267
16, Length: 417823
17, Length: 417823
18, Length: 417895
19, Length: 417895
20, Length: 417895
21, Length: 417895
22, Length: 417895
23, Length: 417967
24, Length: 417967
25, Length: 417967
26, Length: 417967
27, Length: 417967
28, Length: 417967
29, Length: 417967
30, Length: 417967
31, Length: 418059
32, Length: 418059
33, Length: 418059
34, Length: 419533
35, Length: 419533
36, Length: 419533
37, Length: 419533
38, Length: 419533
39, Length: 419743
40, Length: 419743
41, Length: 419743
42, Length: 419743
43, Length: 419743
44, Length: 419743
45, Length: 419743
46, Length: 419743
47, Length: 419743
48, Length: 419743
49, Length: 419743
50, Length: 419743
51, Length: 419743
52, Length: 421309
53, Length: 421309
54, Length: 421309
55, Length: 421309
56, Length: 421309

In [23]:
all_chunks.to_csv('searches_no_dup.csv', sep='^', index=False )

In [24]:
! bzip2 searches_no_dup.csv
! ls -alt

total 1083644
drwxr-x--- 2 dsc dsc      4096 ene 26 12:31 .
-rw-r--r-- 1 dsc dsc  10013467 ene 26 12:30 searches_no_dup.csv.bz2
-rw-r--r-- 1 dsc dsc  56437955 ene 26 12:01 bookings_no_dup.csv.bz2
-rwxrwxrwx 1 dsc dsc    244720 ene 25 17:20 searches_sample.csv.bz2
-rwxrwxrwx 1 dsc dsc   4232732 ene 25 17:19 bookings_sample.csv
-rwxrwxrwx 1 dsc dsc    535893 ene 25 17:19 bookings_sample.csv.bz2
-rwxr-x--- 1 dsc dsc 483188920 mar 13  2018 searches.csv.bz2
-rwxr-x--- 1 dsc dsc 554970628 mar 13  2018 bookings.csv.bz2
drwxr-x--- 7 dsc dsc      4096 mar 13  2018 ..


##  Trabajo con esos ficheros

In [6]:
import pandas as pd

In [7]:
bookings = pd.read_csv('bookings_no_dup.csv.bz2', sep='^', nrows= 1)

In [8]:
bookings.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [9]:
bookings = pd.read_csv('bookings_no_dup.csv.bz2', sep='^', usecols=['dep_port', 'arr_port', 'cre_date           '])

In [10]:
searches = pd.read_csv('searches_no_dup.csv.bz2', sep='^', usecols=['Destination', 'Origin', 'Date'])

In [11]:
bookings.shape, searches.shape

((1016377, 3), (421392, 3))

In [12]:
bookings.sample(5)

,cre_date,dep_port,arr_port
986359,2013-11-04 00:00:00,CVG,ATL
348383,2013-03-06 00:00:00,PLZ,KUL
477780,2013-05-30 00:00:00,BAH,KTM
427200,2013-03-07 00:00:00,PDX,SJC
197295,2013-01-09 00:00:00,YQB,SAF


In [13]:
searches.sample(5)

,Date,Origin,Destination
7747,2013-01-08,PMI,AGP
73427,2013-03-15,MRS,MSY
1465,2013-01-02,LHR,SYD
256295,2013-09-14,BOS,DUB
289792,2013-10-17,CPT,MXP


####  0 -> drop nan, add index

In [14]:
searches.reset_index(inplace=True)
searches.sample(5)

,index,Date,Origin,Destination
283243,283243,2013-10-11,HAM,URT
239462,239462,2013-08-28,HNL,ANC
300535,300535,2013-10-28,BAQ,VLC
274356,274356,2013-10-02,EDI,BAH
137718,137718,2013-05-18,MOW,NYC


In [15]:
searches.dropna(inplace=True)
searches.tail()

,index,Date,Origin,Destination
421387,421387,2013-08-17,GVA,LHR
421388,421388,2013-08-17,HKT,BKK
421389,421389,2013-08-17,PHL,TPA
421390,421390,2013-08-17,ORD,GDL
421391,421391,2013-10-13,VIE,HA


#### 1-> make sure dep_port & arr_port dont have spaces

In [16]:
bookings.head()

,cre_date,dep_port,arr_port
0,2013-02-22 00:00:00,ZRH,LHR
1,2013-03-26 00:00:00,SAL,CLT
2,2013-03-26 00:00:00,SAL,CLT
3,2013-03-26 00:00:00,AKL,SVO
4,2013-03-26 00:00:00,AKL,SVO


In [17]:
bookings.columns = bookings.columns.str.strip()
bookings.columns

Index(['cre_date', 'dep_port', 'arr_port'], dtype='object')

In [18]:
bookings['dep_port'] = bookings['dep_port'].str.strip()

In [19]:
bookings['arr_port'] = bookings['arr_port'].str.strip()

In [20]:
searches.columns

Index(['index', 'Date', 'Origin', 'Destination'], dtype='object')

In [21]:
searches.columns = searches.columns.str.strip()
searches.columns

Index(['index', 'Date', 'Origin', 'Destination'], dtype='object')

In [22]:
searches['Origin'] = searches['Origin'].str.strip()

In [23]:
searches['Destination'] = searches['Destination'].str.strip()

####  2 -> make suce cre_date and search_date have the same format

In [25]:
def has_expected_format(input):
    if(len(input) != 8): return False
    if(input[2] != '-'): return False
    if(input[5] != '-'): return False

    return True

Esto no es lo que había que hacer

In [27]:
def remove_time(input):
    return input[:10]

In [28]:
has_expected_format('8')


False

In [29]:
has_expected_format('08-11-07')

True

In [30]:
has_expected_format('8-11-07')

False

In [31]:
has_expected_format('08-1-07')

False

In [32]:
bookings['cre_date'] = bookings['cre_date'].apply(remove_time)

In [33]:
bookings.tail()

,cre_date,dep_port,arr_port
1016372,2013-03-28,EWR,SEA
1016373,2013-03-28,EWR,SEA
1016374,2013-03-07,EWR,MIA
1016375,2013-03-28,RDM,PHL
1016376,2013-03-28,RDM,PHL


#### 3 -> add 'Booked' column with 1 in bookings

In [34]:
bookings['Booked'] = 1

In [35]:
bookings.tail()

,cre_date,dep_port,arr_port,Booked
1016372,2013-03-28,EWR,SEA,1
1016373,2013-03-28,EWR,SEA,1
1016374,2013-03-07,EWR,MIA,1
1016375,2013-03-28,RDM,PHL,1
1016376,2013-03-28,RDM,PHL,1


#### 4 -> group by dep_port, arr_port, cre_date

In [36]:
group = bookings.groupby(['dep_port','arr_port', 'cre_date'])
group

Hacemos que nos cuente los booked

In [37]:
group = group.sum().reset_index() 
group

,dep_port,arr_port,cre_date,Booked
0,AAB,ROV,2013-05-29,2
1,AAB,ROV,2013-08-24,2
2,AAE,ALG,2013-01-07,1
3,AAE,ALG,2013-01-08,4
4,AAE,ALG,2013-01-09,2
5,AAE,ALG,2013-01-29,1
6,AAE,ALG,2013-02-18,2
7,AAE,ALG,2013-03-10,2
8,AAE,ALG,2013-03-17,6
9,AAE,ALG,2013-03-26,2


#### 5 -> left join searches + Bookeds

In [38]:
bookings_grouped = group

In [40]:
joined = searches.merge(bookings_grouped,how='left', left_on=['Destination', 'Origin', 'Date'], right_on =['arr_port', 'dep_port', 'cre_date'] )

In [41]:
joined.head()

,index,Date,Origin,Destination,dep_port,arr_port,cre_date,Booked
0,0,2013-01-01,TXL,AUH,NaN,NaN,NaN,NaN
1,1,2013-01-01,ATH,MIL,NaN,NaN,NaN,NaN
2,2,2013-01-01,ICT,SFO,NaN,NaN,NaN,NaN
3,3,2013-01-01,RNB,ARN,NaN,NaN,NaN,NaN
4,4,2013-01-01,OSL,MAD,NaN,NaN,NaN,NaN


Es un left join, pueden aparecer vacios de la derecha

In [42]:
joined.drop(['arr_port', 'dep_port', 'cre_date'], axis=1, inplace=True)

In [43]:
joined.head()

,index,Date,Origin,Destination,Booked
0,0,2013-01-01,TXL,AUH,NaN
1,1,2013-01-01,ATH,MIL,NaN
2,2,2013-01-01,ICT,SFO,NaN
3,3,2013-01-01,RNB,ARN,NaN
4,4,2013-01-01,OSL,MAD,NaN


In [45]:
joined[ joined["Booked"] > 1].head()

,index,Date,Origin,Destination,Booked
27,27,2013-01-01,RUH,JED,17.0
40,40,2013-01-01,DMM,MNL,5.0
59,59,2013-01-01,ATL,MIA,6.0
134,134,2013-01-01,MEL,SYD,10.0
172,172,2013-01-01,BOM,JED,2.0


#### 6 -> Change Nan to 0 and if >0 to 1

In [47]:
joined['Booked'].fillna(value=0,inplace=True)

In [48]:
joined.head()

,index,Date,Origin,Destination,Booked
0,0,2013-01-01,TXL,AUH,0.0
1,1,2013-01-01,ATH,MIL,0.0
2,2,2013-01-01,ICT,SFO,0.0
3,3,2013-01-01,RNB,ARN,0.0
4,4,2013-01-01,OSL,MAD,0.0


In [51]:
def is_not_zero(input):
    if input == 0: return 0
    
    return 1

joined['Booked'] = joined['Booked'].apply(is_not_zero)

In [53]:
joined.head()

,index,Date,Origin,Destination,Booked
0,0,2013-01-01,TXL,AUH,0
1,1,2013-01-01,ATH,MIL,0
2,2,2013-01-01,ICT,SFO,0
3,3,2013-01-01,RNB,ARN,0
4,4,2013-01-01,OSL,MAD,0


In [54]:
joined['Booked'].describe()

count    421391.000000
mean          0.035682
std           0.185496
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Booked, dtype: float64

Sabemos que sólo hay unos y ceros

Con la mean, vemos que el 3.5 % de las búsquedas acabó en reserva

#### 7 -> Add column in the file. searched

In [56]:
joined.rename(columns={'index': 'old_index_from_file'}, inplace=True)

In [57]:
joined.head()

,old_index_from_file,Date,Origin,Destination,Booked
0,0,2013-01-01,TXL,AUH,0
1,1,2013-01-01,ATH,MIL,0
2,2,2013-01-01,ICT,SFO,0
3,3,2013-01-01,RNB,ARN,0
4,4,2013-01-01,OSL,MAD,0


In [61]:
searchs = pd.read_csv('searches_no_dup.csv.bz2', sep='^', low_memory=True)

/home/dsc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
searchs.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


Ya del joined sólome interesa el indice antiguo y el booked

In [70]:
booked = joined.drop(['Date', 'Origin', 'Destination'], axis=1)
booked.head()

,old_index_from_file,Booked
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [71]:
searchs_con_bookings = searchs.merge(booked, how='left',left_index= True, right_on='old_index_from_file')
searchs_con_bookings.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,old_index_from_file,Booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,1,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,2,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,3,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,4,0.0


In [73]:
searchs_con_bookings.drop(['old_index_from_file'],axis=1, inplace=True)
searchs_con_bookings.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,Booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,0.0


In [76]:
searchs_con_bookings['Booked'].describe()

count    421391.000000
mean          0.035682
std           0.185496
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Booked, dtype: float64

In [74]:
searchs_con_bookings.to_csv('search_with_bookings.csv', sep='^', index = False)

In [75]:
! head search_with_bookings.csv

Date^Time^TxnCode^OfficeID^Country^Origin^Destination^RoundTrip^NbSegments^Seg1Departure^Seg1Arrival^Seg1Date^Seg1Carrier^Seg1BookingCode^Seg2Departure^Seg2Arrival^Seg2Date^Seg2Carrier^Seg2BookingCode^Seg3Departure^Seg3Arrival^Seg3Date^Seg3Carrier^Seg3BookingCode^Seg4Departure^Seg4Arrival^Seg4Date^Seg4Carrier^Seg4BookingCode^Seg5Departure^Seg5Arrival^Seg5Date^Seg5Carrier^Seg5BookingCode^Seg6Departure^Seg6Arrival^Seg6Date^Seg6Carrier^Seg6BookingCode^From^IsPublishedForNeg^IsFromInternet^IsFromVista^TerminalID^InternetOffice^Booked
2013-01-01^20:25:57^MPT^624d8c3ac0b3a7ca03e3c167e0f48327^DE^TXL^AUH^1.0^2.0^TXL^AUH^2013-01-26^D2^^AUH^TXL^2013-02-02^D2^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^FRA^0.0
2013-01-01^10:15:33^MPT^b0af35b31588dc4ab06d5cf2986e8e02^MD^ATH^MIL^0.0^1.0^ATH^MIL^2013-01-04^^^^^^^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^KIV^0.0
2013-01-01^18:04:49^MPT^3561a60621de06ab1badc8ca55699ef3^US^ICT^SFO^1.0^2.0^ICT^SFO^2013-08-